#### COPY ALL DCM IMAGES INTO FROM stage_2_test_images INTO stage_2_train_images

In [30]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [31]:
root_path = '/home/phoenix/Desktop/ITU/Trustable AI/Project/pneumonia-detection-pytorch-master/'

In [32]:
def get_bb(img_t):
    for rec in train_labels_df[train_labels_df['patientId'] == img_t].values.tolist():
        try:
            return (rec[1], rec[2], rec[3], rec[4])
        except:
            return (None)

In [33]:
balanced_label_df = pd.read_csv(root_path + 'dataset/balanced_label.txt')
balanced_label_df

,ab514814-ddbf-46df-b913-650244f68f48,2
0,5ead2889-223f-4fa3-a540-04495121c121,2
1,987d7028-3873-4b8c-aa45-d3f52f3501ed,2
2,a95a4817-482e-48bd-96bf-c661573e7032,2
3,3d04aef0-2bb8-4de2-a871-4c814bca5d9b,2
4,9d021f29-5ab1-4713-a377-3cd80baaa477,2
...,...,...
26394,e77f1166-ed2e-4cc2-820f-c5c0e3a50abb,1
26395,dec5c8de-c916-441c-a23d-16b8815658d1,1
26396,15c93233-9362-477c-8e4d-c25f2e651aa7,1
26397,afd6c35a-134e-4607-bf71-e5a1c06264c8,1


In [34]:
TRAIN_SAMPLES = 26400
detailed_class_info_df = pd.read_csv(root_path + 'dataset/stage_2_detailed_class_info.csv')
detailed_class_info_df

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [35]:
train_labels_df = pd.read_csv(root_path + 'dataset/stage_2_train_labels.csv')
train_labels_df

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [36]:
detailed_class_info_df = detailed_class_info_df.groupby('class').apply(lambda x: x.sample(TRAIN_SAMPLES//3).reset_index(drop=True))
detailed_class_info_df

patientId         class
class                                                                
Lung Opacity 0     02cfa7d1-61e0-4d9e-8664-9a7469ccab76  Lung Opacity
             1     bf22936e-6390-4f6f-b7c2-9452972c3ab1  Lung Opacity
             2     5f1863c0-7804-419a-878d-b9719f354b22  Lung Opacity
             3     719bb18b-a428-4fd8-9338-46f1187ffa5c  Lung Opacity
             4     cfb88e62-c5d6-415c-87d1-7be3238de1d5  Lung Opacity
...                                                 ...           ...
Normal       8795  56bea3ed-36f7-44ff-bcbd-d6bef00a1c9f        Normal
             8796  5cdd56f1-9ad7-4b45-81e2-6f5c0a890cec        Normal
             8797  e482e77c-d511-4871-b14b-4f912d94ac9c        Normal
             8798  2847bf15-c5f4-47fc-ab15-0fec98933128        Normal
             8799  a5d13986-5ec1-4eab-b8a4-c01d46122a3f        Normal

[26400 rows x 2 columns]

In [37]:
#np.savetxt(r'/home/tianshu/pneumonia/dataset/balanced_label.txt', df.values, fmt='%s')
f = open(root_path + 'dataset/balanced_label.txt', 'w')
arr = detailed_class_info_df.values
new_line = ''
for index in tqdm(range(arr.shape[0])):
    line = arr[index]
    img = line[0]
    label = line[1]
    if label=='No Lung Opacity / Not Normal':
        label = '0'
    elif label=='Normal':
        label = '1'
    elif label=='Lung Opacity':
        label= '2'
    
    new_line = root_path + 'dataset/rsna-pneumonia-detection-challenge/stage_2_train_images/' + img + '.dcm' + ',' + label + '\n'
    f.write(new_line)

f.close()


100%|██████████| 26400/26400 [00:00<00:00, 881661.46it/s]


In [38]:
info_0 = []
info_1 = []
info_2 = []

with open(root_path + 'dataset/balanced_label.txt') as file:
    for line in file:
        cur_line = line.rstrip().split(",")
        if cur_line[1] == '0':         
            info_0.append(cur_line)
        elif cur_line[1] == '1':
            info_1.append(cur_line)
        elif cur_line[1] == '2':
            info_2.append(cur_line)


In [39]:
#info_0_train, info_0_test, _, _ = train_test_split(info_0, info_0, test_size=0.33, random_state=42)
#len(info_0_train), len(info_0_test)

In [40]:
#info_1_train, info_1_test, _, _ = train_test_split(info_1, info_1, test_size=0.33, random_state=42)
#len(info_1_train), len(info_1_test)

In [41]:
#info_2_train, info_2_test, _, _ = train_test_split(info_2, info_2, test_size=0.33, random_state=42)
#len(info_2_train), len(info_2_test)

In [42]:
!pwd

/home/phoenix/Desktop/ITU/Trustable AI/Project/pneumonia-detection-pytorch-master


In [43]:
def splitData(data_list0, data_list1, data_list2, n_splits=3):
    def getChunk(data_list, indexes):
        return_list = []
        for i in indexes:
            return_list.append(data_list[i])
        return return_list
    
    def writeFile(data, data_name):
        textfile = open(data_name, "w")
        for element in tqdm(data):
            names_seperated_slash = str(element[0]).split('/')
            patient_id = names_seperated_slash[len(names_seperated_slash)-1][:-4]
            element.append(get_bb(patient_id))
            textfile.write(str(element) + "\n")
        textfile.close()        
        
    kf = KFold(n_splits=n_splits)
    for fold_no, (train_index, test_index) in enumerate(kf.split(data_list0)): # doesn't matter if;
        # it is data_list0 or data_list1 or data_list0. because we use it only for getting indexes.
        #print("FOLD NO: ", fold_no, "TRAIN:", train_index, "TEST:", test_index)
        X_train_0 = getChunk(data_list0, train_index)
        X_test_0 = getChunk(data_list0, test_index)
        X_train_1 = getChunk(data_list1, train_index)
        X_test_1 = getChunk(data_list1, test_index)
        X_train_2 = getChunk(data_list2, train_index)
        X_test_2 = getChunk(data_list2, test_index)
        
        X_train = X_train_0 + X_train_1 + X_train_2
        X_test = X_test_0 + X_test_1 + X_test_2
        
        writeFile(X_train, "dataset/X_train_fold_" + str(fold_no) + ".txt")
        writeFile(X_test, "dataset/X_test_fold_" + str(fold_no) + ".txt")
            

In [44]:
splitData(info_0, info_1, info_2)

100%|██████████| 8799/8799 [00:12<00:00, 694.86it/s]
